In [41]:
# !pip install yfinance

import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf

In [42]:
df = pd.read_csv('stockerbot-export.csv', error_bad_lines = False)
# There are 12 lines with more than 8 fields. I will remove it for now since the counts are small
df.head()

b'Skipping line 731: expected 8 fields, saw 13\nSkipping line 2836: expected 8 fields, saw 15\nSkipping line 3058: expected 8 fields, saw 12\nSkipping line 3113: expected 8 fields, saw 12\nSkipping line 3194: expected 8 fields, saw 17\nSkipping line 3205: expected 8 fields, saw 17\nSkipping line 3255: expected 8 fields, saw 17\nSkipping line 3520: expected 8 fields, saw 17\nSkipping line 4078: expected 8 fields, saw 17\nSkipping line 4087: expected 8 fields, saw 17\nSkipping line 4088: expected 8 fields, saw 17\nSkipping line 4499: expected 8 fields, saw 12\n'


,id,text,timestamp,source,symbols,company_names,url,verified
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True
2,1019711413798035500,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,True
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,True
4,1019718460287389700,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,True


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28264 entries, 0 to 28263
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             28264 non-null  int64 
 1   text           28264 non-null  object
 2   timestamp      28264 non-null  object
 3   source         28264 non-null  object
 4   symbols        28264 non-null  object
 5   company_names  28263 non-null  object
 6   url            21895 non-null  object
 7   verified       28264 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 1.5+ MB


In [44]:
# Split Timestamp Column into Dates and times
df[['dayofweek','month','day','time','timezone', 'year']] = df.timestamp.str.split(expand=True)
df[['hour','minute','second']] = df.time.str.split(':',expand=True)
df.head()

,id,text,timestamp,source,symbols,company_names,url,verified,dayofweek,month,day,time,timezone,year,hour,minute,second
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True,Wed,Jul,18,21:33:26,+0000,2018,21,33,26
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True,Wed,Jul,18,22:22:47,+0000,2018,22,22,47
2,1019711413798035500,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,True,Wed,Jul,18,22:32:01,+0000,2018,22,32,01
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,True,Wed,Jul,18,22:52:52,+0000,2018,22,52,52
4,1019718460287389700,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,True,Wed,Jul,18,23:00:01,+0000,2018,23,00,01


In [45]:
# Split company names by '*' except for E*TRADE Financial Corporation
com = []
for i in df.company_names.astype('str'):
    if i == 'E*TRADE Financial Corporation':
        com.append([i])
    else:
        com.append(i.split('*'))
df['company_names'] = com

# Split symbols by '-'
df['symbols'] = df['symbols'].str.split("-")

df.head()

,id,text,timestamp,source,symbols,company_names,url,verified,dayofweek,month,day,time,timezone,year,hour,minute,second
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,[GS],[The Goldman Sachs],https://twitter.com/i/web/status/1019696670777...,True,Wed,Jul,18,21:33:26,+0000,2018,21,33,26
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,[M],[Macy's],https://twitter.com/i/web/status/1019709091038...,True,Wed,Jul,18,22:22:47,+0000,2018,22,22,47
2,1019711413798035500,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,[AIG],[American],https://buff.ly/2L3kmc4,True,Wed,Jul,18,22:32:01,+0000,2018,22,32,01
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,[BTC],[Bitcoin],https://twitter.com/i/web/status/1019716662587...,True,Wed,Jul,18,22:52:52,+0000,2018,22,52,52
4,1019718460287389700,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,[ORCL],[Oracle],http://on.forbes.com/6013DqDDU,True,Wed,Jul,18,23:00:01,+0000,2018,23,00,01


In [46]:
# exlode the rows for `symbols` and `company_names` with multiple values
df = pd.concat([df[x].explode() for x  in  ['symbols','company_names']], axis=1).\
    join(df.loc[:, ~df.columns.isin(['symbols', 'company_names'])]).reindex(columns=df.columns)
df['Date'] = pd.to_datetime(df['timestamp']).dt.date
df.head()

,id,text,timestamp,source,symbols,company_names,url,verified,dayofweek,month,day,time,timezone,year,hour,minute,second,Date
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,True,Wed,Jul,18,21:33:26,+0000,2018,21,33,26,2018-07-18
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,True,Wed,Jul,18,22:22:47,+0000,2018,22,22,47,2018-07-18
2,1019711413798035500,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,True,Wed,Jul,18,22:32:01,+0000,2018,22,32,01,2018-07-18
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,True,Wed,Jul,18,22:52:52,+0000,2018,22,52,52,2018-07-18
4,1019718460287389700,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,True,Wed,Jul,18,23:00:01,+0000,2018,23,00,01,2018-07-18


In [47]:
stock_symbols = df[['symbols','company_names']].drop_duplicates().reset_index(drop = True)
stock_symbols.head(20)

,symbols,company_names
0,GS,The Goldman Sachs
1,M,Macy's
2,AIG,American
3,BTC,Bitcoin
4,ORCL,Oracle
5,FB,Facebook
6,GOOGL,Alphabet
7,GOOG,Alphabet
8,HRS,Harris
9,INTC,Intel


In [48]:
# # Clean stock symbols data
# This process take around 1 hour to retreive the data. I saved data to the csv file 'stock_symbols.csv' for convenient for usage
# temp_name = []
# a = -1
# for i in stock_symbols['symbols']:
#     try:
#         stock = yf.Ticker(i)
#         stock.info['longName']
#     except:
#         temp_name.append('Error')
#     else:
#         temp_name.append(stock.info['longName'])
#     a = a+1
#     print(a)
# temp_name
# stock_symbols['longName'] = temp_name
# stock_symbols.to_csv('stock_symbols.csv')


In [49]:
stock_symbols = pd.read_csv('stock_symbols.csv',index_col= 0)
stock_symbols = stock_symbols.fillna('0')
stock_symbols.head()

,symbols,company_names,longName
0,GS,The Goldman Sachs,"The Goldman Sachs Group, Inc."
1,M,Macy's,"Macy's, Inc."
2,AIG,American,"American International Group, Inc."
3,BTC,Bitcoin,ClearShares Piton Intermediate Fixed Income ETF
4,ORCL,Oracle,Oracle Corporation


In [50]:
# Exclude symbols that are not stock / out of market / merged with other companies
temp_isStock = []
for i in range(len(stock_symbols)):
    c, l = stock_symbols.iloc[i][['company_names','longName']]
    cut_name = ' '.join([i for i in l.split(' ') if i not in ['International', 'Group,', 'Inc.','plc','Corporation']])
    if l == 'Error' or l == '0':
        temp_isStock.append(False)
    elif c == l:
        temp_isStock.append(True)
    elif c == ' '.join([i for i in l.split(' ') if i not in ['International', 'Group,', 'Inc.','plc','Corporation']]):
        temp_isStock.append(True)
    elif c == cut_name[:-1]:
        temp_isStock.append(True)
    else:
        temp_isStock.append(False)
        
stock_symbols_c1 = pd.concat([stock_symbols,pd.Series(temp_isStock)], axis = 1)
stock_symbols_c1.columns = ['symbols', 'company_names', 'longName','is_stock']
stock_symbols_c1.head()

,symbols,company_names,longName,is_stock
0,GS,The Goldman Sachs,"The Goldman Sachs Group, Inc.",True
1,M,Macy's,"Macy's, Inc.",True
2,AIG,American,"American International Group, Inc.",True
3,BTC,Bitcoin,ClearShares Piton Intermediate Fixed Income ETF,False
4,ORCL,Oracle,Oracle Corporation,True


In [51]:
# There are 53 stocks that cannot match the name, I used eye-scan for these. I excluded only 'BTC' and 'S' from the stock
stock_symbols_c1.loc[(stock_symbols_c1['is_stock'] == False)
              &(~(stock_symbols_c1['longName'].isin(['Error','0']))
              & ~(stock_symbols_c1['symbols'].isin(['BTC','S'])))
              ,'is_stock'] = True

stock_symbols_c1.head(20)

,symbols,company_names,longName,is_stock
0,GS,The Goldman Sachs,"The Goldman Sachs Group, Inc.",True
1,M,Macy's,"Macy's, Inc.",True
2,AIG,American,"American International Group, Inc.",True
3,BTC,Bitcoin,ClearShares Piton Intermediate Fixed Income ETF,False
4,ORCL,Oracle,Oracle Corporation,True
5,FB,Facebook,"Meta Platforms, Inc.",True
6,GOOGL,Alphabet,Alphabet Inc.,True
7,GOOG,Alphabet,Alphabet Inc.,True
8,HRS,Harris,Error,False
9,INTC,Intel,Intel Corporation,True


In [52]:
# For checking is_stock == False
# stock_symbols_c1[stock_symbols_c1['is_stock'] == False]

In [53]:
stock_symbols_cleaned = stock_symbols_c1[stock_symbols_c1['is_stock'] == True].reset_index(drop = True)

stock_symbols_cleaned = stock_symbols_cleaned.drop_duplicates('symbols').reset_index(drop = True)

In [54]:
print(df['timestamp'].min())
print(df['timestamp'].max())

Fri Feb 23 18:50:19 +0000 2018
Wed Jul 18 23:46:27 +0000 2018


In [55]:
# Stock Price from Feb 23 to Jul 31, 2018. I saved this data to CSV. Running this code might take around 30 mins
# stock_price = pd.DataFrame(columns= ['symbols', 'Date', 'Close'])
# a = -1
# error = []
# for i in stock_symbols_cleaned['symbols']:
#     try:
#         pdr.get_data_yahoo(i, start="2018-02-23", end="2018-07-31").reset_index()[['Date','Close']]
#     except:
#         error.append(i)
#     else:
#         temp_df = pdr.get_data_yahoo(i, start="2018-02-23", end="2018-07-31").reset_index()[['Date','Close']]
#         temp_df['symbols'] = i
#         temp_df = temp_df[['symbols','Date','Close']]
#         stock_price = pd.concat([stock_price,temp_df],ignore_index=True)
#     a = a+1
#     print(a, end = ' ')
    
# stock_price
# stock_price.to_csv('stock_price.csv')
# pd.DataFrame(error,columns= ['Error']).to_csv('Error.csv')

In [56]:
stock_price = pd.read_csv('stock_price.csv')
stock_price['Date'] = pd.to_datetime(stock_price.Date)

**Caution:**

COL: does not have data for 2018-05-21 and 2018-07-02.

ARRY, FMSA, XL, MON, DPS, FOX, HTZ, GGP, FOXA, CHK, ABX: These companies are delisted. (11 stocks)

PAH: Changed symbol to 'ESI'

ETE: Changed symbol to 'ET'

VRX: Changed symbol to 'BHC'

ARNC: Changed symbol to 'HWM'

In [57]:
# Change symbol in original df
change = [('PAH','ESI'),('ETE','ET'),('VRX','BHC'),('ARNC','HWM')]
          
for o,n in change:
    df.loc[df['symbols'] == o,'symbols'] = n
    stock_symbols_cleaned.loc[stock_symbols_cleaned['symbols'] == o,'symbols'] = n

In [58]:
# Check no of stock in file stock price
print(stock_price['symbols'].nunique())
print(stock_symbols_cleaned['symbols'].nunique() - 11)

396
396


In [59]:
# Create new data only including id, date, source, symbols, dayofweek from df
stockerbot = pd.DataFrame()
stockerbot['id'] = df.id
stockerbot['Date'] = pd.to_datetime(df.Date)
stockerbot['source'] = df.source
stockerbot['symbols'] = df.symbols
stockerbot['dayofweek'] = df.dayofweek
stockerbot.head()

,id,Date,source,symbols,dayofweek
0,1019696670777503700,2018-07-18,GoldmanSachs,GS,Wed
1,1019709091038548000,2018-07-18,StockTwits,M,Wed
2,1019711413798035500,2018-07-18,TheStreet,AIG,Wed
3,1019716662587740200,2018-07-18,MarketWatch,BTC,Wed
4,1019718460287389700,2018-07-18,Forbes,ORCL,Wed


In [60]:
# merge stockerbot and price data
combined_data = stockerbot.merge(stock_price, on = ['symbols', 'Date'], how = 'outer')
combined_data.head()

,id,Date,source,symbols,dayofweek,Unnamed: 0,Close
0,1.019697e+18,2018-07-18,GoldmanSachs,GS,Wed,100.0,231.240005
1,1.019709e+18,2018-07-18,StockTwits,M,Wed,210.0,37.700001
2,1.019711e+18,2018-07-18,TheStreet,AIG,Wed,320.0,55.130001
3,1.019380e+18,2018-07-18,ThisLincolnian,AIG,Wed,320.0,55.130001
4,1.019452e+18,2018-07-18,ConsumerFeed,AIG,Wed,320.0,55.130001


In [61]:
# stockerbot.to_csv('stockerbot.csv')
# price.to_csv('price1.csv')
# combined_data.to_csv('combined_data.csv')